In [ ]:
#Make sure your using the correct environment from UWGDA2021 and the kernel 
#matches that environment in the above menu
# I downlaod all the DEMS and shapefiles to my disk and then call them into jupyter notebook. I don't fuss
#with API codes and such
#THere were many tribulations in this process. One of which was learning again what could be masked, warped, 
#clipped and reprojected. I don't have those answers, but maybe most. This code can provide a less cluttered
#workflow and should be coupled with lab 07 from uwgda2021 as the authoratative reference.
#
#This should do the trick or at least drastically speed up a potentially long process

In [ ]:
import os
import requests
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.markers as mmarkers
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio import plot, mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import rasterstats
from shapely.geometry import Point, Polygon, MultiPolygon

In [ ]:
%matplotlib widget
#%matplotlib inline

In [ ]:
wa_bounds = (-124.733174, 45.543541, -116.915989, 49.002494) # or other bounds too, but maybe they can be 
#gotten later from the shapefile that represents your area of interest

In [ ]:
#Download your DEM (digital elevation model) of interest to the 
# working directory
# 'input.tif' is the DEM you downloaded. We are warping it to fit our coordinate system...I think 
#this is COMMAND LINE code
gdalwarp -t_srs EPSG:4326 input.tif output.tif



In [ ]:
# Load your reprojected dataset

with rio.open(output.tif) as src_proj:
    rio.plot.show(src_proj)
    
    #or 
    
src_proj = rio.open('output.tif')

In [ ]:
#Now take all that warped elevation data and generate a hillshade using COMMAND LINE in your working
#directory

gdaldem hillshade path/src_proj.tif path/hs_fn.tif

In [ ]:
#load hillshade
hs_fn = 'hs_fn.tif'

hs_src = rio.open(hs_fn)
hs = hs_src.read(1, masked=True)

#or try this if you have problems
hs = rio.open('hs_fn')

In [ ]:
# get hs extent
hs_extent = rio.plot.plotting_extent(hs_src)
hs_extent

In [ ]:
#  check it out your hillshade. Pretty cool...if this worked so far.
f, ax1 = plt.subplots()
ax1.imshow(hs, cmap='gray', extent=hs_extent);

In [ ]:
# A NOTE: I cam across a problem where I needed to figure out the order files could be manipulated and it can
#be confusing. If this order is not working I suggest referring to lab 07

In [ ]:
#Now we have a (1) DEM and (2) a hillshade

In [ ]:
#setting up my MORA park shapefile (which is a geopandas df)
parks = 'parks.shp'
parks_gdf = gpd.read_file(parks)

#reproject to you location
dst_crs = 'EPSG:32610'
parks_gdf_proj = parks_gdf.to_crs(dst_crs)

#Now I isolate MORA geometry in gdf because this gpd has EVERY park
mora = parks_gdf_proj.loc[parks_gdf_proj['UNIT_CODE'] == 'MORA']
mora_geom = mora.iloc[0].geometry

In [ ]:
#NOw we generate masked arrays for MORA
#these are the keyword arguments for masking your arrays (DEMS-->masked arrays)
rio_mask_kwargs = {'filled':False, 'crop':True, 'indexes':1}

# *** [mora_geom] has brackets because there's only 1 polygon. ditch brackets if the geometry is a multipolygon
mora_ma, mora_ma_transform = rio.mask.mask(src_proj, [mora_geom], **rio_mask_kwargs)

In [ ]:
#Download shape files for your area of interest, roads, wateways, lakes, geological stuff, etc.

In [ ]:
#load shapefiles as geodataframes and reproject
#Load as GeoDataFrame and reproject
water_bodies_shp_fn = 'WA_Hydrography_-_NHDWaterbody.shp'
water_bodies_gdf = gpd.read_file(water_bodies_shp_fn)
water_bodies_gdf_utm = water_bodies_gdf.to_crs(src_proj.crs)

polygons_shp_fn = 'mora_nvum_polygons.shp'
polygons_gdf = gpd.read_file(polygons_shp_fn)
polygons_gdf_utm = polygons_gdf.to_crs(src_proj.crs);

roads_shp_fn = 'NPS_-_Roads_-_Geographic_Coordinate_System.shp'
roads_gdf = gpd.read_file(roads_shp_fn)
roads_gdf_utm = roads_gdf.to_crs(src_proj.crs)

trails_shp_fn = 'NPS_-_Trails_-_Web_Mercator.shp'
trails_gdf = gpd.read_file(trails_shp_fn)
trails_gdf_utm = trails_gdf.to_crs(src_proj.crs)

flowline_shp_fn = 'WA_Hydrography_-_NHD_Flowline.shp'
flowline_gdf = gpd.read_file(flowline_shp_fn)
flowline_gdf_utm = flowline_gdf.to_crs(src_proj.crs)

In [ ]:
#clip shapefiles to MORA
mora_water_bodies_gdf_utm = gpd.clip(water_bodies_gdf_utm, mora_geom, keep_geom_type=False)
mora_polygons_gdf_utm = gpd.clip(polygons_gdf_utm, mora_geom, keep_geom_type=False)
roads_gdf_utm = gpd.clip(roads_gdf_utm, mora_geom, keep_geom_type=False)
counters_gdf_utm = gpd.clip(counters_gdf_utm, mora_geom, keep_geom_type=False)
trails_gdf_utm = gpd.clip(trails_gdf_utm, mora_geom, keep_geom_type=False)
flowline_gdf_utm = gpd.clip(flowline_gdf_utm, mora_geom, keep_geom_type=False)

In [ ]:
#1. we should have a DEM clipped to MORA - this provides elevation data for color bars and stuff
#2. We has a hilllshade (masked array...i beleive) clipped to MORA to give a realistic appearance
#3. All of the shape files we need are clipped and reprojected to fit MORA Boundaries
#4. Then the legend

In [ ]:
#plot all of the layers to make a map

f, axa = plt.subplots(figsize=(40,10))

#elevation (DEM) and hillshade (masked array)
m = axa.imshow(mora_ma, cmap='gist_earth', vmin=500, vmax= 2000,extent=mora_ma_extent, alpha=0.2, zorder=1)
plt.colorbar(m, ax=axa,label='Elevation (m)', shrink=0.2, aspect=5)
axa.imshow(hs, cmap='gray', extent=mora_ma_extent, alpha=0.5)

#lake names and red dots
texts = []
for x, y, label in zip(lake_names.centroid.x, lake_names.centroid.y, lake_names["GNIS_Name"]):
    texts.append(plt.text(x, y, label, ha='center', va='top', weight='bold', fontsize = 12))
#lake_names.centroid.plot(ax=axa, color='firebrick', marker='o',markersize=20, zorder=1.5)

#shape files
mora_water_bodies_gdf_utm.plot(ax=axa, color='dodgerblue')
mora_polygons_gdf_utm.plot(ax=axa, color = 'yellow', alpha=0.3)
mora_polygons_gdf_utm.exterior.plot(ax=axa, color = 'red', linewidth=0.2, alpha=1)
counters_gdf_utm.plot(ax=axa, color='blue', marker='x',markersize=25,zorder=1.3)
roads_gdf_utm.plot(ax=axa, color='black', linewidth=1, zorder=1)
trails_gdf_utm.plot(ax=axa, color='brown', linestyle='dashed', linewidth=0.7, zorder=1)
flowline_gdf_utm.plot(ax=axa, color='dodgerblue', linewidth=0.5, zorder=1)

#legend
axa.add_artist(ScaleBar(1.0))
yellow_patch = mpatches.Patch(color='yellow', ec='red', label='Social media zones', alpha=0.3)
ir =  mlines.Line2D([], [],color='blue', linestyle='None', marker='x',label='IR counters')
lks =  mlines.Line2D([], [],color='firebrick', linestyle='None', marker='o',label='Target lakes')
stream = mlines.Line2D([], [], color='dodgerblue', label='Streams and rivers')
trl = mlines.Line2D([], [], linestyle= '--',color='brown', label='Trails')
roads = mlines.Line2D([], [],color='black', label='Major roads')
axa.legend(loc='lower right',handles=[yellow_patch, stream, trl, roads, ir, lks])

#plt.legend(['IR counters'], loc='center left')
plt.show()